<a href="https://colab.research.google.com/github/piaosan78/TensorFlow_2.0_basic/blob/main/transfer_model_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 텐서 gpu 인스톨

In [39]:
!pip install tensorflow-gpu==2.10.0

## tqdm 인스톨

In [40]:
!pip install tqdm

## 데이터셋 압축파일 다운

In [41]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2023-09-19 06:54:20--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.215.207, 173.194.216.207, 173.194.217.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.215.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   193MB/s    in 0.3s    

2023-09-19 06:54:21 (193 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## 라이브러리 소환

In [42]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.10.0'

## 압출 풀기

In [43]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [44]:
zip_object = zipfile.ZipFile(file = dataset_path, mode="r")

In [45]:
zip_object.extractall("./")

In [46]:
zip_object.close()

## 데이터셋 경로 설정

In [47]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [48]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## 모델 구축

### 전이모델 (모바일넷 V2)

In [49]:
IMG_SHAPE = (128, 128, 3)

In [50]:
base_model = tf.keras.applications.MobileNetV2(input_shape = IMG_SHAPE, include_top=False, weights="imagenet")

In [51]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_2[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][

### 기본 모델 동결

In [52]:
base_model.trainable = False

### 커스텀 헤드 추가

In [53]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [54]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [55]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d_1')>

In [56]:
prediction_layer = tf.keras.layers.Dense(units = 1, activation = 'sigmoid')(global_average_layer)

### 모델 추가

In [57]:
model = tf.keras.models.Model(inputs = base_model.input, outputs = prediction_layer)

In [58]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_2[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']         

### 모델 컴파일

In [72]:
from tensorflow.keras.optimizers import Adam

# optimizers = Adam(learning_rate =0.00001)

model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])


### 제네레이터 생성

In [60]:
data_gen_train = ImageDataGenerator(rescale = 1/255.)
data_gen_valid = ImageDataGenerator(rescale = 1/255.)

In [35]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size = (128,128), batch_size = 128, class_mode = "binary")

Found 2000 images belonging to 2 classes.


In [36]:
valid_generator = data_gen_train.flow_from_directory(train_dir, target_size = (128,128), batch_size = 128, class_mode = "binary")

Found 2000 images belonging to 2 classes.


## 모델 훈련

In [73]:
model.fit(train_generator, epochs= 5 , validation_data = valid_generator)

Epoch 1/5
16/16 [==============================] - 9s 411ms/step - loss: 0.4307 - accuracy: 0.7955 - val_loss: 0.2318 - val_accuracy: 0.9155
Epoch 2/5
16/16 [==============================] - 5s 348ms/step - loss: 0.1869 - accuracy: 0.9285 - val_loss: 0.1527 - val_accuracy: 0.9400
Epoch 3/5
16/16 [==============================] - 5s 348ms/step - loss: 0.1394 - accuracy: 0.9435 - val_loss: 0.1259 - val_accuracy: 0.9510
Epoch 4/5
16/16 [==============================] - 5s 345ms/step - loss: 0.1203 - accuracy: 0.9540 - val_loss: 0.1110 - val_accuracy: 0.9570
Epoch 5/5
16/16 [==============================] - 5s 348ms/step - loss: 0.1070 - accuracy: 0.9595 - val_loss: 0.1005 - val_accuracy: 0.9600


## 전이학습 모델 평가

In [74]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

<ipython-input-74-a86cc1f7f1e2>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)


In [75]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.9599999785423279


## 파인 튜닝 시작

### 동결 코드 해제

In [76]:
base_model.trainable = True

In [77]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 154


In [78]:
fine_tune_at = 50

In [79]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### 컴파일 파인 튜닝

In [80]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


### 파인 튜닝 시동

In [81]:
model.fit(train_generator,
          epochs=5,
          validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 15s 446ms/step - loss: 0.1531 - accuracy: 0.9405 - val_loss: 0.0376 - val_accuracy: 0.9865
Epoch 2/5
16/16 [==============================] - 6s 354ms/step - loss: 0.0175 - accuracy: 0.9990 - val_loss: 0.0278 - val_accuracy: 0.9895
Epoch 3/5
16/16 [==============================] - 6s 351ms/step - loss: 0.0069 - accuracy: 1.0000 - val_loss: 0.0314 - val_accuracy: 0.9880
Epoch 4/5
16/16 [==============================] - 6s 357ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.0205 - val_accuracy: 0.9930
Epoch 5/5
16/16 [==============================] - 6s 350ms/step - loss: 9.8551e-04 - accuracy: 1.0000 - val_loss: 0.0186 - val_accuracy: 0.9945


### 파인 튜닝 모델 평가

In [82]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

<ipython-input-82-a86cc1f7f1e2>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)


In [83]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.9944999814033508
